In [1]:
import pandas as pd
import os
from time import sleep
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support.ui import Select
from selenium.common.exceptions import TimeoutException
from selenium.common.exceptions import StaleElementReferenceException
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.common.by import By

In [2]:
CHROMEDRIVER_PATH = './chromedriver'
WINDOW_SIZE = "1920,1080"

In [3]:
# Comman function for init driver
def init_driver(url):
    driver = webdriver.Chrome(executable_path=CHROMEDRIVER_PATH)
    driver.maximize_window()
    driver.get(url)
    return driver

In [4]:
def wait_for_page_load(xpath_url):
    try:
        element_present = EC.presence_of_element_located((By.XPATH, xpath_url))
        WebDriverWait(driver, 5).until(element_present)
    except TimeoutException:
        print("Timed out waiting for page to load")
    except NoSuchElementException:
        print("Element not found")

In [5]:
product_ids = [
    'B07BS4TJ43',
#     'B07RJWB548',
]
# 6 records from each section total 54
five_ratings = [
    'five_star',
#     'four_star',
#     'three_star',
#     'two_star',
#     'one_star'
]
# https://www.amazon.in/product-reviews/B07BS4TJ43/

In [6]:
driver = init_driver('https://google.com')

In [17]:
final_data = [] 
def fetch_all_data():
    product_count = 0  
    while product_count < 5:
        sleep(5)
        # wait for page load
        wait_for_page_load('//div[@data-hook="review"]')    

        # finding top all boxes 
        all_box = driver.find_elements_by_xpath('//div[@data-hook="review"]') 

        # Ittrating over box to find details
        for one_box in all_box:

            rating = one_box.find_element_by_xpath('.//i/span[@class="a-icon-alt"]').get_attribute('innerHTML')
            review_summary = one_box.find_element_by_xpath('.//*[@data-hook="review-title"]').text
            full_review = one_box.find_element_by_xpath('.//span[@data-hook="review-body"]').text



            final_data.append({
                'rating':rating,
                'review_summary':review_summary,
                'full_review':full_review,
            })

            product_count += 1
            if product_count > 5:
                break
                
            
        # Go to next page
        sleep(5)
        try:
            driver.find_element_by_xpath('//li[@class="a-last"]').click()
        except Exception:
            break

    job_data = pd.DataFrame(final_data)
    job_data

In [18]:
for one_product_id in product_ids:    
    review_url = f'https://www.amazon.in/product-reviews/{one_product_id}/'
 
    # Goting to review page
    driver.get(review_url)
    
    # Setting Which ratting to be fetched
    for one_ratting in five_ratings:
        print(one_ratting)
        Select(driver.find_element_by_id('star-count-dropdown')).select_by_value(one_ratting)
        sleep(2)
        
        fetch_all_data()


five_star


In [20]:
job_data = pd.DataFrame(final_data)
job_data

,rating,review_summary,full_review
0,5.0 out of 5 stars,"Good camera for beginners, must try it!",Good camera for beginners.
1,5.0 out of 5 stars,My first canon ....,Great camera. Perfect product for young buddin...
2,5.0 out of 5 stars,Best basic level dslr mainly for wireless remo...,Really impressed with Amazon surprise for deli...
3,5.0 out of 5 stars,"What can I say...brothers it's ""canon""!",I am really fully content with the product. It...
4,5.0 out of 5 stars,Begginers choice,This camera is a perfect choice if you are a b...
5,5.0 out of 5 stars,Nice Product,Good Product who are really have passion for p...


In [21]:
job_data.to_csv('scraped_data.csv')